In [ ]:
!pip install langchain openai chromadb tiktoken

In [98]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.prompt import PromptTemplate
from langchain.schema.document import Document

In [99]:
from langchain.document_loaders import TextLoader
loader = TextLoader("devfest_info.txt")
documents = loader.load()

##Method 1

In [100]:
chunk_size = 200
overlap = 50
input_string = documents[0].page_content
metadata = documents[0].metadata
documents = []

for i in range(0, len(input_string), chunk_size - overlap):
    chunk = input_string[i:i + chunk_size]
    document = Document(page_content=chunk,metadata =metadata)
    documents.append(document)

##Method 2

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

In [101]:
api_key = "API key"


embeddings = OpenAIEmbeddings(openai_api_key=api_key)
vectorstore = Chroma.from_documents(documents, embeddings)

In [102]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [103]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
At the beginning of standalone question add this 'You're a chatbot designed to answer questions about Devfest hackathon.'.
At the end of standalone question add this 'Answer the question with short, energetic, and happy-sounding responses and add emojis.
If the question is unrelated to the context reply with 'I am sorry, I am not CHAT GPT !',
and if you can't find the information, reply with 'I'm sorry, there is no information about it right now'.'
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [104]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(openai_api_key=api_key,temperature=0), vectorstore.as_retriever(), memory=memory, condense_question_prompt=CONDENSE_QUESTION_PROMPT)

In [ ]:
query = "When will DevFest be held?"
result = qa({"question": query})
print(result["answer"])

In [ ]:
query = "who was the winner of the last edition ?"
result = qa({"question": query})
print(result["answer"])

In [ ]:
query = "What is your favorite book and why?"
result = qa({"question": query})
print(result["answer"])

##Test script:

In [ ]:
while True:
  query = input("Question: ")
  result = qa({"question": query})
  print("Response: "+result["answer"])